In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [30]:
def prepare_data(file): 
    data = pd.read_csv(file)

    feat_raw = data['quizzes']
    label_raw = data['solutions']

    feat = []
    label = []

    # Reshape puzzles to 9x9 array
    for i in feat_raw:
        x = np.array([int(j) for j in i]).reshape((9,9,1))
        feat.append(x)
    
    # Normalize values
    feat = np.array(feat)
    feat = feat/9
    feat -= .5    
    
    # Reshape solutions to 81x1 array
    for i in label_raw:
        x = np.array([int(j) for j in i]).reshape((81,1)) - 1
        label.append(x)   
    
    label = np.array(label)
    
    # Remove raw data from memory
    del(feat_raw)
    del(label_raw)    

    # Split to 95% training and 5% validation
    # 5% validation is enough, since it is still 450'000 validations
    x_train, x_test, y_train, y_test = train_test_split(feat, label, test_size=0.20)
    
    return x_train, x_test, y_train, y_test


In [66]:
import keras
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape, Dropout

def get_model():

    model = keras.models.Sequential()

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

    model.add(Flatten())
    model.add(Dense(81*9))
    model.add(Reshape((-1, 9)))
    model.add(Activation('softmax'))
    
    return model
 

## Load the data

In [32]:
x_train, x_test, y_train, y_test = prepare_data('sudoku.csv')

## Train your own Model

In [70]:
from keras.callbacks import Callback, ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=3,
    verbose=1,
    min_lr=1e-6
)
callbacks_list = [reduce_lr]

In [74]:
model = get_model()

adam = keras.optimizers.Adam(learning_rate=.001)
#
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=adam, 
              metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_test,y_test), batch_size=640, epochs=5, verbose=1, callbacks=callbacks_list)

model.save("sudoku.model")

Epoch 1/5


2022-07-17 14:48:37.585447: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1249/1250 [============================>.] - ETA: 0s - loss: 0.9185 - accuracy: 0.8299

2022-07-17 14:49:16.060738: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1250/1250 [==============================] - 44s 35ms/step - loss: 0.9182 - accuracy: 0.8301 - val_loss: 0.4958 - val_accuracy: 0.9732 - lr: 0.0010
Epoch 2/5
1250/1250 [==============================] - 47s 37ms/step - loss: 0.4753 - accuracy: 0.9781 - val_loss: 0.4661 - val_accuracy: 0.9809 - lr: 0.0010
Epoch 3/5
1250/1250 [==============================] - 47s 38ms/step - loss: 0.4526 - accuracy: 0.9860 - val_loss: 0.4518 - val_accuracy: 0.9859 - lr: 0.0010
Epoch 4/5
1250/1250 [==============================] - 47s 38ms/step - loss: 0.4368 - accuracy: 0.9928 - val_loss: 0.4412 - val_accuracy: 0.9899 - lr: 0.0010
Epoch 5/5
1250/1250 [==============================] - 47s 38ms/step - loss: 0.4236 - accuracy: 0.9992 - val_loss: 0.4351 - val_accuracy: 0.9921 - lr: 0.0010


INFO:tensorflow:Assets written to: sudoku.model/assets


INFO:tensorflow:Assets written to: sudoku.model/assets
